In [ ]:
!pip install --upgrade transformers
!pip install simpletransformers

     |████████████████████████████████| 1.5MB 21.3MB/s 
     |████████████████████████████████| 2.9MB 49.2MB/s 
     |████████████████████████████████| 890kB 53.0MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893261 sha256=9fbe09d27121e35f646ac4eb8c29f84d10ab78ba8b173dbc03f5c349eb71c80c
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses
     |████████████████████████████████| 204kB 16.6MB/s 
     |████████████████████████████████| 71kB 10.2MB/s 
     |████████████████████████████████| 1.8MB 55.9MB/s 
     |████████████████████████████████| 7.4MB 51.7MB/s 
     |████████████████████████████████| 1.1MB 50.6MB/s 
     |████████████████████████████████| 51kB 8.5MB/s 
     |████████████████████████████████| 317kB 49.7MB/s 
     |████████████████████████████████| 133kB 54.4MB/s 
     |████████████████████████████████| 163kB 50.6MB/s 
     |███████████████████████████

In [ ]:
# importing important libraries
# checking if the libraries are all good,or is missing thing!
# used try/except to handle it!

try:
  from sklearn.metrics import f1_score
  from sklearn.model_selection import train_test_split
  from simpletransformers.classification import ClassificationModel, ClassificationArgs
  import pandas as pd
  import numpy as np
  import seaborn as sns
  sns.set_theme(style="darkgrid")

except Exception as e:
  print('Package Error! \n\n {}'.format(e))

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#EDA
# calling the train/test data, just to make everything easy!
train = pd.read_csv('/content/drive/MyDrive/hahakathon/train.csv')
test  = pd.read_csv('/content/drive/MyDrive/hahakathon/public_dev.csv')

train.head()

,id,text,is_humor,humor_rating,humor_controversy,offense_rating
0,1,TENNESSEE: We're the best state. Nobody even c...,1,2.42,1.0,0.2
1,2,A man inserted an advertisement in the classif...,1,2.50,1.0,1.1
2,3,How many men does it take to open a can of bee...,1,1.95,0.0,2.4
3,4,Told my mom I hit 1200 Twitter followers. She ...,1,2.11,1.0,0.0
4,5,Roses are dead. Love is fake. Weddings are bas...,1,2.78,0.0,0.1


In [ ]:
train['humor_controversy'].isnull().sum()

3068

In [ ]:
train['is_humor'].value_counts()

1    4932
0    3068
Name: is_humor, dtype: int64

In [ ]:
train.drop(['offense_rating','humor_rating'], axis=1, inplace=True)

In [ ]:
train

,id,text,is_humor,humor_controversy
0,1,TENNESSEE: We're the best state. Nobody even c...,1,1.0
1,2,A man inserted an advertisement in the classif...,1,1.0
2,3,How many men does it take to open a can of bee...,1,0.0
3,4,Told my mom I hit 1200 Twitter followers. She ...,1,1.0
4,5,Roses are dead. Love is fake. Weddings are bas...,1,0.0
...,...,...,...,...
7995,7996,Lack of awareness of the pervasiveness of raci...,0,NaN
7996,7997,Why are aspirins white? Because they work sorry,1,0.0
7997,7998,"Today, we Americans celebrate our independence...",1,0.0
7998,7999,How to keep the flies off the bride at an Ital...,1,0.0


In [ ]:
train.isnull().sum()

id                   0
text                 0
is_humor             0
humor_controversy    0
dtype: int64

In [ ]:
for i in range(len(train)):
  if train['humor_controversy'][i]!= 1 and train['humor_controversy'][i]!= 0:
    train['humor_controversy'][i]=train['is_humor'][i]



In [ ]:
train.drop(['is_humor'], axis=1, inplace=True)

In [ ]:
train['humor_controversy'].value_counts()

0.0    5535
1.0    2465
Name: humor_controversy, dtype: int64

In [ ]:
train.dropna(axis=0, inplace=True)

In [ ]:
train['text']=train['text'].str.lower()

In [ ]:
#train['humor_controversy'].fillna(0, inplace = True) 

In [ ]:
train.isnull().sum()

id                   0
text                 0
humor_controversy    0
dtype: int64

In [ ]:
train.head(5)

,id,text,humor_controversy
0,1,TENNESSEE: We're the best state. Nobody even c...,1.0
1,2,A man inserted an advertisement in the classif...,1.0
2,3,How many men does it take to open a can of bee...,0.0
3,4,Told my mom I hit 1200 Twitter followers. She ...,1.0
4,5,Roses are dead. Love is fake. Weddings are bas...,0.0


In [ ]:
train['humor_controversy'].isnull().sum()

0

In [ ]:
# init the train columns!
train.columns = ['id','text', 'labels']
print(train.columns)

Index(['id', 'text', 'labels'], dtype='object')


In [ ]:
# declaring the train_df  and valid_df values!
train_df, valid_df = train_test_split(train,test_size=0.05 ,stratify=train['labels'], random_state=10)

In [ ]:
#Model args and parameters to start the traring process!!
model_args = ClassificationArgs(num_train_epochs=2, learning_rate=1e-5, overwrite_output_dir=True, manual_seed=17, silent=True)
model = ClassificationModel(model_type='xlnet', model_name='xlnet-large-cased', use_cuda=True, num_labels=2, args=model_args)

Downloading:   0%|          | 0.00/761 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.44G [00:00<?, ?B/s]

Some weights of the model checkpoint at xlnet-large-cased were not used when initializing XLNetForSequenceClassification: ['lm_loss.weight', 'lm_loss.bias']
- This IS expected if you are initializing XLNetForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLNetForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of XLNetForSequenceClassification were not initialized from the model checkpoint at xlnet-large-cased and are newly initialized: ['sequence_summary.summary.weight', 'sequence_summary.summary.bias', 'logits_proj.weight', 'logits_proj.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions

Downloading:   0%|          | 0.00/798k [00:00<?, ?B/s]

In [ ]:
# training the model, with the train set!
model.train_model(train_df)

/usr/local/lib/python3.6/dist-packages/torch/optim/lr_scheduler.py:136: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  "https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate", UserWarning)
/usr/local/lib/python3.6/dist-packages/torch/optim/lr_scheduler.py:216: UserWarning: Please also save or load the state of the optimizer when saving or loading the scheduler.
  warnings.warn(SAVE_STATE_WARNING, UserWarning)


(1900, 0.529632700296413)

In [ ]:
# extracting the results,
# and then starting the presictions with the test set!!
result, model_outputs, wrong_predictions = model.eval_model(valid_df)
print(f'Results: {result}')

Results: {'mcc': 0.39595160927849754, 'tp': 79, 'tn': 213, 'fp': 64, 'fn': 44, 'eval_loss': 0.449160028398037}


In [ ]:
# the predictions step(f1-score),
# here we predicting the output using the model_output and valid_df!!
predictions = []
for x in model_outputs:
    predictions.append(np.argmax(x))

print(f"f1-score: %{round(f1_score(valid_df['labels'], predictions), 4)}")

f1-score: %0.594


In [ ]:
# here we calc the  (precision, recall, accuracy), by using those equations!
# Precision = TP/(TP+FP), Sensitivity(recall)=TP/(TP+FN), Accuracy=(TP+TN)/(TP+TN+FP+FN)

res = []
for key in result.keys():
  res.append(result[key])
# print(f'All keys: {res}')

TP = res[1]
TN = res[2]
FP = res[3]
FN = res[4]
# print(f'TP:{TP}, TN:{TN}, FP:{FP}, FN:{FN}')

precision = TP / (TP+FP)
recall = TP / (TP+FN)
accuracy = (TP+TN) / (TP+TN+FP+FN)
print(f"Precision: %{round(precision, 4)} \nRecall: %{round(recall, 4)} \nAccuracy: %{round(accuracy, 4)}")

Precision: %0.5524 
Recall: %0.6423 
Accuracy: %0.73


In [ ]:
#predicting using the test set!
test_predictions, raw_outputs = model.predict(test['text'])

In [ ]:
# saving the result to .csv path file,
# to submit it on codalab!!
id = test['id']
humor_controversy = test_predictions
submit = pd.DataFrame({'id': id, 'humor_controversy':humor_controversy})
submit.to_csv('c.csv', index=False)

In [ ]:
import zipfile
#declaring the submition,
# and writing to zip file!
submition = zipfile.ZipFile("c.zip", mode='w', compression=zipfile.ZIP_DEFLATED)
submition.write("c.csv")
 
submition.close()